# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = None, names = columns))


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop('area', axis=1)  # Features
y = fires_dt['area']  # Target



# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


preproc1 = ColumnTransformer([
    ('num', StandardScaler(), ['coord_x', 'coord_y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']),
    ('cat', OneHotEncoder(), ['month', 'day'])
])


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [6]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np


def nonlinear_transform(X):
    X_transformed = np.log1p(X)  # Applying log transformation
    return X_transformed


preproc2 = ColumnTransformer([
    ('num', FunctionTransformer(func=nonlinear_transform, validate=False), ['temp']),  # Apply transformation to 'temp'
    ('num_scaled', StandardScaler(), ['coord_x', 'coord_y', 'FFMC', 'DMC', 'DC', 'ISI', 'RH', 'wind', 'rain']),
    ('cat', OneHotEncoder(), ['month', 'day'])
])


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define regressors
baseline_regressor = KNeighborsRegressor()
advanced_regressor = RandomForestRegressor()

# Define pipelines
pipeline_A = Pipeline(steps=[
    ('preproc', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline(steps=[
    ('preproc', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline(steps=[
    ('preproc', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline(steps=[
    ('preproc', preproc2),
    ('regressor', advanced_regressor)
])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



print("Missing values in X_train:\n", X_train.isnull().sum())
print("Missing values in y_train:\n", y_train.isnull().sum())


numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns


preproc1 = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])

preproc2 = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])


baseline_regressor = KNeighborsRegressor()
advanced_regressor = RandomForestRegressor()


pipeline_A = Pipeline(steps=[
    ('preproc', preproc1),
    ('regressor', baseline_regressor)
])

pipeline_B = Pipeline(steps=[
    ('preproc', preproc2),
    ('regressor', baseline_regressor)
])

pipeline_C = Pipeline(steps=[
    ('preproc', preproc1),
    ('regressor', advanced_regressor)
])

pipeline_D = Pipeline(steps=[
    ('preproc', preproc2),
    ('regressor', advanced_regressor)
])


param_grid_A = {
    'regressor__n_neighbors': [3, 5, 7, 10],
    'regressor__weights': ['uniform', 'distance']
}

param_grid_B = {
    'regressor__n_neighbors': [3, 5, 7, 10],
    'regressor__weights': ['uniform', 'distance']
}

param_grid_C = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

param_grid_D = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}


grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error', verbose=1)


grid_A.fit(X_train, y_train)
grid_B.fit(X_train, y_train)
grid_C.fit(X_train, y_train)
grid_D.fit(X_train, y_train)


print("Best parameters for Pipeline A:", grid_A.best_params_)
print("Best score for Pipeline A:", grid_A.best_score_)

print("Best parameters for Pipeline B:", grid_B.best_params_)
print("Best score for Pipeline B:", grid_B.best_score_)

print("Best parameters for Pipeline C:", grid_C.best_params_)
print("Best score for Pipeline C:", grid_C.best_score_)

print("Best parameters for Pipeline D:", grid_D.best_params_)
print("Best score for Pipeline D:", grid_D.best_score_)


Missing values in X_train:
 coord_x    0
coord_y    0
month      0
day        0
FFMC       0
DMC        0
DC         0
ISI        0
temp       0
RH         0
wind       0
rain       0
dtype: int64
Missing values in y_train:
 0
Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\User

Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\User

Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\User

Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\User

Best parameters for Pipeline A: {'regressor__n_neighbors': 3, 'regressor__weights': 'uniform'}
Best score for Pipeline A: nan
Best parameters for Pipeline B: {'regressor__n_neighbors': 3, 'regressor__weights': 'uniform'}
Best score for Pipeline B: nan
Best parameters for Pipeline C: {'regressor__max_depth': None, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 50}
Best score for Pipeline C: nan
Best parameters for Pipeline D: {'regressor__max_depth': None, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 50}
Best score for Pipeline D: nan


# Evaluate

+ Which model has the best performance?

In [10]:
# Print the best score for each pipeline
print("Best score for Pipeline A:", grid_A.best_score_)
print("Best score for Pipeline B:", grid_B.best_score_)
print("Best score for Pipeline C:", grid_C.best_score_)
print("Best score for Pipeline D:", grid_D.best_score_)

# Determine which pipeline has the best score
best_score = max(grid_A.best_score_, grid_B.best_score_, grid_C.best_score_, grid_D.best_score_)

if best_score == grid_A.best_score_:
    print("Pipeline A has the best performance.")
elif best_score == grid_B.best_score_:
    print("Pipeline B has the best performance.")
elif best_score == grid_C.best_score_:
    print("Pipeline C has the best performance.")
elif best_score == grid_D.best_score_:
    print("Pipeline D has the best performance.")
else:
    print("There was an error in determining the best performing pipeline.")


Best score for Pipeline A: nan
Best score for Pipeline B: nan
Best score for Pipeline C: nan
Best score for Pipeline D: nan
There was an error in determining the best performing pipeline.


# Export

+ Save the best performing model to a pickle file.

I was not able to find it and do not know what was the error in the previous code. 

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(best_pipeline.named_steps['regressor'].fit(X_train, y_train))

# Compute SHAP values for the selected observation
shap_values = explainer(X_test[idx])


# Summary plot of SHAP values for the selected observation
shap.summary_plot(shap_values, X_test[idx], feature_names=X_test.columns)


# Summary plot of mean absolute SHAP values across all observations
shap.summary_plot(shap_values, X_train, feature_names=X_train.columns)


*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.